<a href="https://colab.research.google.com/github/HandanYU/Rumour-detection/blob/main/obtain_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. split source reply ids

In [10]:
from utils import timer

In [11]:
@timer('ms')
def split_source_reply(txt_file):
  """
  txt_file: 'train.data.txt'
  """
  with open(txt_file) as f:
      ids = f.readlines()
  source_ids = []
  reply_ids = []
  source_txt_file = txt_file.split('.')[0] + '_source_data.txt'
  reply_txt_file = txt_file.split('.')[0] + '_reply_data.txt'
  for i in range(len(ids)):
      source_ids.append(ids[i].split(',')[0].strip())
      reply_ids.extend([r.strip() for r in ids[i].split(',')[1:]])
# save source_ids
  with open(source_txt_file,'w') as f:
      for i in source_ids:
          f.write(i)
          f.write('\n')
# save reply_ids
  with open(reply_txt_file,'w') as f:
      for i in reply_ids:
        f.write(i)
        f.write('\n')
# split_source_reply('dev.data.txt')

# 2. Crawl tweet

In [ ]:
# !twarc2 hydrate  dev.txt > dev_reply_data.jsonl
# resource: https://scholarslab.github.io/learn-twarc/06-twarc-command-basics.html

# 3. Get tweet features

In [12]:
from collections import defaultdict
import json
import pandas as pd

@timer('ms')
def filter_feature(jsonl_file_name):
    """
    jsonl_file_name: 'dev_source_data.jsonl'
    """
    json_file_name = '_'.join(jsonl_file_name.split('_')[:-1]) + '.json'
    json_data = pd.read_json(path_or_buf=jsonl_file_name, lines=True)
    data_dict = defaultdict(dict)
    for i in range(json_data.shape[0]):
        for j in range(len(json_data.data.iloc[i])):
            data_dict[json_data.data.iloc[i][j]['id']]['text'] = json_data.data.iloc[i][j]['text']
            data_dict[json_data.data.iloc[i][j]['id']]['reply_count'] = json_data.data.iloc[i][j]['public_metrics']['reply_count']
            data_dict[json_data.data.iloc[i][j]['id']]['like_count'] = json_data.data.iloc[i][j]['public_metrics']['like_count']
            data_dict[json_data.data.iloc[i][j]['id']]['retweet_count'] = json_data.data.iloc[i][j]['public_metrics']['retweet_count']
            data_dict[json_data.data.iloc[i][j]['id']]['quote_count'] = json_data.data.iloc[i][j]['public_metrics']['quote_count']
            data_dict[json_data.data.iloc[i][j]['id']]['possibly_sensitive'] = json_data.data.iloc[i][j]['possibly_sensitive']
            data_dict[json_data.data.iloc[i][j]['id']]['created_at'] = json_data.data.iloc[i][j]['created_at'] #add create time
            data_dict[json_data.data.iloc[i][j]['id']]['user_id'] = json_data.data.iloc[i][j]['author_id'] #add user id
            data_dict[json_data.data.iloc[i][j]['id']]['has_url'] = 1 if 'entities' in json_data.data.iloc[i][j] else 0 #add shared url number
            

    #  convert into json format
    dict_json=json.dumps(data_dict)
    # save json file
    with open(json_file_name, 'w+') as file:
        file.write(dict_json)
# filter_feature('dev_reply_data.jsonl')

### dump data

In [13]:
jsonls = ['./data/full data/dev_source_data.jsonl',
          './data/full data/dev_reply_data.jsonl',
          './data/full data/train_source_data.jsonl',
          './data/full data/train_reply_data.jsonl']

for jsonl in jsonls:
    filter_feature(jsonl)

Call filter_feature in 203.33385467529297 ms
Call filter_feature in 1865.3488159179688 ms
Call filter_feature in 418.25294494628906 ms
Call filter_feature in 5743.713140487671 ms


## 4. Get user info
#### currently only implement on Source tweet

In [14]:
@timer('ms')
def get_user_info(jsonl_file_name):
    """
    jsonl_file_name: 'dev_source_data.jsonl'
    """
    json_file_name = '_'.join(jsonl_file_name.split('_')[:-1]) + '_userinfo.json'
    json_data = pd.read_json(path_or_buf=jsonl_file_name, lines=True)
    # collect the user info
    info_dict = defaultdict(dict)
    for i in range(json_data.shape[0]):
        for j in range(len(json_data.includes.iloc[i]['users'])):
            info_dict[json_data.includes.iloc[i]['users'][j]['id']]['followers_count'] = json_data.includes.iloc[i]['users'][j]['public_metrics']['followers_count']
            info_dict[json_data.includes.iloc[i]['users'][j]['id']]['tweet_count'] = json_data.includes.iloc[i]['users'][j]['public_metrics']['tweet_count']
            info_dict[json_data.includes.iloc[i]['users'][j]['id']]['verified'] = json_data.includes.iloc[i]['users'][j]['verified']
    #  convert into json format
    dict_json=json.dumps(info_dict)
    # save json file
    with open(json_file_name, 'w+') as file:
        file.write(dict_json)
    
jsonls = ['./data/full data/dev_source_data.jsonl',
          './data/full data/train_source_data.jsonl']

for jsonl in jsonls:
    get_user_info(jsonl)

Call get_user_info in 132.65705108642578 ms
Call get_user_info in 304.43310737609863 ms
